## DPENet

#### Import module

In [1]:
import glob
import json
import os
import numpy as np
import math
from tqdm.notebook import trange, tqdm
from time import sleep
import torch
import torch.nn as nn
from torch import optim
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from tensorboardX import SummaryWriter

#### Load Input and GT

GT format  
- ['ego_global_x', 'ego_global_y']  

Input format  
- ['ego_vx', 'yaw_rate', 'sas_angle', 'long_acc', 'lat_acc']

In [2]:
gt_folder = './DDPE_GT_adaf_1/'
in_folder = './DDPE_input_adaf_1/'
gt_file_list = glob.glob(gt_folder+'/*_gt.json')
gt_file_list.sort()
in_train, gt_train = [], []
in_test, gt_test = [], []

In [3]:
# Load the json log.
for k in trange(len(gt_file_list),desc='folder progress'):
    # Load the GT
    file_name_gt = gt_file_list[k]
    with open(file_name_gt, 'r', encoding='utf-8-sig') as data_file_gt:
        try:
            data_gt = json.load(data_file_gt)
        except:
            continue
    
    # Load the input
    file_name_in = in_folder + gt_file_list[k][17:26] + '_in.json'
    with open(file_name_in, 'r', encoding='utf-8-sig') as data_file_in:
        try:
            data_in = json.load(data_file_in)
        except:
            continue
            
    if len(data_gt['ego_global_y']) < 30:
        continue
        
    if k % 10 == 0:
        gt_test.append([data_gt['ego_global_x'], data_gt['ego_global_y']])
        in_test.append([data_in['ego_vx'],data_in['yaw_rate'],data_in['sas_angle'],data_in['long_acc'],data_in['lat_acc']])
    else:
        gt_train.append([data_gt['ego_global_x'], data_gt['ego_global_y']])
        in_train.append([data_in['ego_vx'],data_in['yaw_rate'],data_in['sas_angle'],data_in['long_acc'],data_in['lat_acc']])

folder progress:   0%|          | 0/533745 [00:00<?, ?it/s]

In [4]:
# make list to tensor
gt_train = torch.tensor(gt_train).cuda()
in_train = torch.tensor(in_train).cuda()
gt_test = torch.tensor(gt_test).cuda()
in_test = torch.tensor(in_test).cuda()

In [5]:
# class to use the dataloader
class DPEDataset(data.Dataset):
    def __len__(self):
        return len(in_train)
    
    def __getitem__(self,idx):
        in_tmp = in_train[idx]
        gt_tmp = gt_train[idx]
        return {'in':in_tmp,'gt':gt_tmp}

In [6]:
class DPETestset(data.Dataset):
    def __len__(self):
        return len(in_test)
    
    def __getitem__(self,idx):
        in_tm = in_test[idx]
        gt_tm = gt_test[idx]
        return {'in':in_tm,'gt':gt_tm}

In [7]:
dpe_dataset = DPEDataset()
dpe_testset = DPETestset()

In [8]:
len(gt_train), len(in_train), len(gt_test), len(in_test)

(480370, 480370, 53375, 53375)

In [ ]:
# Exercise

#### Loss function

In [9]:
# get lateral point from the polynomial equation
# input 
# : longitudinal distance, C1, C2, C3
# output 
# : lateral point x

def get_x_polynomial(long_dist, C1, C2, C3):
    mean = 0.0
    inv_sigma = 1.0
    y_ = (long_dist-mean) * inv_sigma
    
    x = C1 * (y_**1) + C2 * (y_**2) + C3 * (y_**3)
    
    return x

In [10]:
# MSELoss from lateral point
# input
# : [C1, C2, C3], gt['ego_global_x', 'ego_global_y']
# output
# : sum, variance, standard deviation of MSELoss

def lp_loss(output, gt):
    criterion = nn.MSELoss()
    # criterion = nn.CrossEntropyLoss()
    mse_total = []
    mse_mean, mse_var, mse_std = 0.0, 0.0, 0.0
    for i in range(len(gt[0][1])):
        x_tmp = get_x_polynomial(gt[0][1][i],output[0][0],output[0][1],output[0][2])
        # make to tensor if needed
        x_tmp_torch = x_tmp
        gt_tmp_torch = gt[0][0][i]
        
        mse_tmp = criterion(x_tmp_torch,gt_tmp_torch)
        mse_total.append(mse_tmp)
    mse_mean = torch.mean(torch.tensor(mse_total))
    mse_var = torch.var(torch.tensor(mse_total))
    mse_std = torch.std(torch.tensor(mse_total))
    
    return mse_mean, mse_var, mse_std

In [ ]:
# Exercise

#### DPENet

In [11]:
# DPENet
# input
# : CAN information (ego_vx, yaw_rate, sas_angle, long_acc, lat_acc)
# output
# : variable for polynomial equation (C1, C2, C3)

# construct model on cuda if available
use_cuda = torch.cuda.is_available()

class DPENet(nn.Module):
    def __init__(self):
        super(DPENet,self).__init__()
        # fullyconncted layer
        fc1 = nn.Linear(5,64,bias=True)
        fc2 = nn.Linear(64,128,bias=True)
        fc3 = nn.Linear(128,256,bias=True)
        fc4 = nn.Linear(256,512,bias=True)
        fc5 = nn.Linear(512,512,bias=True)
        fc6 = nn.Linear(512,512,bias=True)
        fc7 = nn.Linear(512,512,bias=True)
        fc8 = nn.Linear(512,3,bias=True)
        tanh = nn.Tanh()
        dropout = nn.Dropout(0.3)
        #self.relu = nn.ReLU()
        
        self.fc_module = nn.Sequential(
            fc1,
            tanh,
            dropout,
            fc2,
            tanh,
            dropout,
            fc3,
            tanh,
            dropout,
            fc4,
            tanh,
            dropout,
            fc5,
            tanh,
            dropout,
            fc6,
            tanh,
            dropout,
            fc7,
            tanh,
            dropout,
            fc8
        )
        # use gpu
        if use_cuda:
            self_fc_module = self.fc_module.cuda()
        
    def forward(self,x):
        out = self.fc_module(x)
        
        return out

In [12]:
print(DPENet())

DPENet(
  (fc_module): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=64, out_features=128, bias=True)
    (4): Tanh()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=128, out_features=256, bias=True)
    (7): Tanh()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=256, out_features=512, bias=True)
    (10): Tanh()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): Tanh()
    (14): Dropout(p=0.3, inplace=False)
    (15): Linear(in_features=512, out_features=512, bias=True)
    (16): Tanh()
    (17): Dropout(p=0.3, inplace=False)
    (18): Linear(in_features=512, out_features=512, bias=True)
    (19): Tanh()
    (20): Dropout(p=0.3, inplace=False)
    (21): Linear(in_features=512, out_features=3, bias=True)
  )
)


In [ ]:
# Exercise

In [ ]:
# history (w_dropout_wo_loss_scaling)
        self.fc_module = nn.Sequential(
            fc1,
            tanh,
            dropout,
            fc2,
            tanh,
            dropout,
            fc3,
            tanh,
            dropout,
            fc4,
            tanh,
            dropout,
            fc5,
            tanh,
            dropout,
            fc6,
            tanh,
            dropout,
            fc7,
            tanh,
            dropout,
            fc8,
            tanh,
            dropout,
            fc9,
            tanh,
            dropout,
            fc10
        )

#### Train

In [13]:
# Xavier Normal initialization
def weight_init(m): 
    if isinstance(m, nn.Linear):
        size = m.weight.size()
        fan_out = size[0] # number of rows
        fan_in = size[1] # number of columns
        variance = np.sqrt(2.0/(fan_in + fan_out))
        m.weight.data.normal_(0.0, variance)

In [14]:
# argument
# learning rate and beta1,2 for optimizer
dpenet_learning_rate = 0.01 # 0.01(x) 0.001 0.0002
dpenet_momentum = 0.9
dpenet_beta1 = 0.5
dpenet_beta2 = 0.999
# lambda for loss
lambda_mean = 10**-7
lambda_var = 10**-10
lambda_std = 10**-7
# number of epoch and log steo
num_epochs = 1000
log_step = 1000

In [ ]:
# dataloader
data_loader = data.DataLoader(dataset=dpe_dataset, batch_size=1, shuffle=True)
test_loader = data.DataLoader(dataset=dpe_testset, batch_size=1)

# tensorboardX summary
summary = SummaryWriter('log')
dpenet = DPENet()
dpenet.apply(weight_init)

# To make graph in summary
dummmy = torch.zeros(5)
summary.add_graph(dpenet,dummmy.cuda())

# Optimizer
dpenet_optimizer = optim.Adam(dpenet.parameters(), lr=dpenet_learning_rate)
#dpenet_optimizer = optim.Adam(dpenet.parameters(), dpenet_learning_rate, [dpenet_beta1, dpenet_beta2])
#dpenet_optimizer = optim.SGD(dpenet.parameters(), lr=dpenet_learning_rate, momentum=dpenet_momentum)

for epoch in range(num_epochs):
    for i, sample in enumerate(data_loader):
        
        # input : ['ego_vx', 'yaw_rate', 'sas_angle', 'long_acc', 'lat_acc']
        # gt : ['ego_global_x', 'ego_global_y']
        input_ = sample['in'].cuda()
        gt_ = sample['gt']
        output_ = dpenet(input_)
        #print(output_)
        # train dpenet
        # reset the gradient for the optimizer
        # zero_grad : 역전파 실행 전 변화도 0으로
        dpenet_optimizer.zero_grad()
        
        # loss from lateral position
        loss_lp_mean, loss_lp_var, loss_lp_std = lp_loss(output_,gt_)
        loss_lp_mean = lambda_mean*loss_lp_mean
        loss_lp_var = lambda_var*loss_lp_var
        loss_lp_std = lambda_std*loss_lp_std
        loss = loss_lp_mean + loss_lp_var + loss_lp_std
        loss = Variable(loss_lp_mean, requires_grad=True)
        # backpropagate the gradient
        loss.backward()
        # update the weights using the gradient with the optimizer
        dpenet_optimizer.step()
        
        # print the log information
        if (i+1) % log_step == 0:
            print('Epoch [%d/%d], BatchStep[%d/%d]' % (epoch + 1, num_epochs, i + 1, len(in_train)))
            print('Loss: %.4f, lp_mean: %.4f, lp_var: %.4f, lp_std: %.4f' 
                  % (loss.data, loss_lp_mean.data, loss_lp_var.data, loss_lp_std.data))

    loss_test = 0
    loss_test_lsit = []
    with torch.no_grad():
        for i, sample_t in enumerate(test_loader):
            input_t = sample_t['in'].cuda()
            gt_t = sample_t['gt']
            output_t = dpenet(input_t)
            loss_lp_mean_t, loss_lp_var_t, loss_lp_std_t = lp_loss(output_t,gt_t)
            loss_test_lsit.append(loss_lp_mean_t)
            
    loss_test = torch.mean(torch.tensor(loss_test_lsit))
    print('Epoch [%d/%d]' % (epoch + 1, num_epochs))
    print('Test Loss: %.4f' % (loss_test.data))

    # visualize the parameter
    for name, param in dpenet.named_parameters():
        #summary.add_histogram(name, param, epoch)
        summary.add_histogram(name, param.clone().cpu().data.numpy(), epoch, bins='doane')
    
    # save summary
    summary.add_scalar('loss/loss', loss.item(), epoch)
    summary.add_scalar('loss/loss_lp_mean', loss_lp_mean.item(), epoch)
    summary.add_scalar('loss/loss_lp_var', loss_lp_var.item(), epoch)
    summary.add_scalar('loss/loss_lp_std', loss_lp_std.item(), epoch)
    summary.add_scalar('loss_t/loss', loss_test.item(), epoch)
    
    summary.flush()
    # save the model parameters
    #if epoch % 10 == 0:
    model_path = os.path.join('./DPENet_model', 'DPENet-%d.pkl' % (epoch + 1))
    torch.save(dpenet.state_dict(), model_path)

Epoch [1/1000], BatchStep[1000/480370]
Loss: 149.2882, lp_mean: 149.2882, lp_var: 616340736.0000, lp_std: 248.2621
Epoch [1/1000], BatchStep[2000/480370]
Loss: 13956.6699, lp_mean: 13956.6699, lp_var: 5560322228224.0000, lp_std: 23580.3340
Epoch [1/1000], BatchStep[3000/480370]
Loss: 35460.6133, lp_mean: 35460.6133, lp_var: 35211781865472.0000, lp_std: 59339.5195
Epoch [1/1000], BatchStep[4000/480370]
Loss: 2846.3879, lp_mean: 2846.3879, lp_var: 211504250880.0000, lp_std: 4598.9595
Epoch [1/1000], BatchStep[5000/480370]
Loss: 0.5858, lp_mean: 0.5858, lp_var: 5841.9868, lp_std: 0.7643
Epoch [1/1000], BatchStep[6000/480370]
Loss: 16.2695, lp_mean: 16.2695, lp_var: 4928003.0000, lp_std: 22.1991
Epoch [1/1000], BatchStep[7000/480370]
Loss: 1657.0956, lp_mean: 1657.0956, lp_var: 78691753984.0000, lp_std: 2805.2051
Epoch [1/1000], BatchStep[8000/480370]
Loss: 6.3030, lp_mean: 6.3030, lp_var: 1123101.2500, lp_std: 10.5976
Epoch [1/1000], BatchStep[9000/480370]
Loss: 17.1285, lp_mean: 17.1285,

Epoch [1/1000], BatchStep[72000/480370]
Loss: 1158.4550, lp_mean: 1158.4550, lp_var: 37282066432.0000, lp_std: 1930.8564
Epoch [1/1000], BatchStep[73000/480370]
Loss: 0.0000, lp_mean: 0.0000, lp_var: 0.0000, lp_std: 0.0000
Epoch [1/1000], BatchStep[74000/480370]
Loss: 711.2313, lp_mean: 711.2313, lp_var: 13770609664.0000, lp_std: 1173.4824
Epoch [1/1000], BatchStep[75000/480370]
Loss: 0.7585, lp_mean: 0.7585, lp_var: 17458.7344, lp_std: 1.3213
Epoch [1/1000], BatchStep[76000/480370]
Loss: 0.0000, lp_mean: 0.0000, lp_var: 0.0000, lp_std: 0.0000
Epoch [1/1000], BatchStep[77000/480370]
Loss: 11645.4668, lp_mean: 11645.4668, lp_var: 3741348003840.0000, lp_std: 19342.5645
Epoch [1/1000], BatchStep[78000/480370]
Loss: 1738.4381, lp_mean: 1738.4381, lp_var: 82259124224.0000, lp_std: 2868.0852
Epoch [1/1000], BatchStep[79000/480370]
Loss: 0.0000, lp_mean: 0.0000, lp_var: 0.0000, lp_std: 0.0000
Epoch [1/1000], BatchStep[80000/480370]
Loss: 3183.0088, lp_mean: 3183.0088, lp_var: 284272689152.000

Epoch [1/1000], BatchStep[143000/480370]
Loss: 127690.2812, lp_mean: 127690.2812, lp_var: 455769849856000.0000, lp_std: 213487.6719
Epoch [1/1000], BatchStep[144000/480370]
Loss: 36160.7227, lp_mean: 36160.7227, lp_var: 36183438524416.0000, lp_std: 60152.6719
Epoch [1/1000], BatchStep[145000/480370]
Loss: 517.2499, lp_mean: 517.2499, lp_var: 7711257088.0000, lp_std: 878.1376
Epoch [1/1000], BatchStep[146000/480370]
Loss: 24847.0762, lp_mean: 24847.0762, lp_var: 16534340632576.0000, lp_std: 40662.4414
Epoch [1/1000], BatchStep[147000/480370]
Loss: 11.7723, lp_mean: 11.7723, lp_var: 4126728.7500, lp_std: 20.3144
Epoch [1/1000], BatchStep[148000/480370]
Loss: 2448.7593, lp_mean: 2448.7593, lp_var: 167261487104.0000, lp_std: 4089.7615
Epoch [1/1000], BatchStep[149000/480370]
Loss: 0.0000, lp_mean: 0.0000, lp_var: 0.0000, lp_std: 0.0000
Epoch [1/1000], BatchStep[150000/480370]
Loss: 846.3918, lp_mean: 846.3918, lp_var: 18520023040.0000, lp_std: 1360.8829
Epoch [1/1000], BatchStep[151000/480

Epoch [1/1000], BatchStep[214000/480370]
Loss: 45461.6172, lp_mean: 45461.6172, lp_var: 59533024034816.0000, lp_std: 77157.6406
Epoch [1/1000], BatchStep[215000/480370]
Loss: 245.8580, lp_mean: 245.8580, lp_var: 1738196736.0000, lp_std: 416.9169
Epoch [1/1000], BatchStep[216000/480370]
Loss: 34.3235, lp_mean: 34.3235, lp_var: 42235244.0000, lp_std: 64.9887
Epoch [1/1000], BatchStep[217000/480370]
Loss: 0.1308, lp_mean: 0.1308, lp_var: 423.7755, lp_std: 0.2059
Epoch [1/1000], BatchStep[218000/480370]
Loss: 2086.9121, lp_mean: 2086.9121, lp_var: 122714939392.0000, lp_std: 3503.0693
Epoch [1/1000], BatchStep[219000/480370]
Loss: 503.1339, lp_mean: 503.1339, lp_var: 5379842560.0000, lp_std: 733.4741
Epoch [1/1000], BatchStep[220000/480370]
Loss: 3558.9709, lp_mean: 3558.9709, lp_var: 357560385536.0000, lp_std: 5979.6353
Epoch [1/1000], BatchStep[221000/480370]
Loss: 872.0591, lp_mean: 872.0591, lp_var: 20416575488.0000, lp_std: 1428.8658
Epoch [1/1000], BatchStep[222000/480370]
Loss: 74557

Epoch [1/1000], BatchStep[284000/480370]
Loss: 0.3286, lp_mean: 0.3286, lp_var: 2430.9346, lp_std: 0.4930
Epoch [1/1000], BatchStep[285000/480370]
Loss: 2212.1484, lp_mean: 2212.1484, lp_var: 142531428352.0000, lp_std: 3775.3337
Epoch [1/1000], BatchStep[286000/480370]
Loss: 1853.1294, lp_mean: 1853.1294, lp_var: 99193266176.0000, lp_std: 3149.4963
Epoch [1/1000], BatchStep[287000/480370]
Loss: 0.0402, lp_mean: 0.0402, lp_var: 78.6133, lp_std: 0.0887
Epoch [1/1000], BatchStep[288000/480370]
Loss: 62528.8164, lp_mean: 62528.8164, lp_var: 105920319193088.0000, lp_std: 102917.6016
Epoch [1/1000], BatchStep[289000/480370]
Loss: 18071.3574, lp_mean: 18071.3574, lp_var: 8806418350080.0000, lp_std: 29675.6094
Epoch [1/1000], BatchStep[290000/480370]
Loss: 0.0061, lp_mean: 0.0061, lp_var: 0.3399, lp_std: 0.0058
Epoch [1/1000], BatchStep[291000/480370]
Loss: 7833.1465, lp_mean: 7833.1465, lp_var: 1703284703232.0000, lp_std: 13050.9951
Epoch [1/1000], BatchStep[292000/480370]
Loss: 17.8375, lp_m

...

Epoch [6/1000], BatchStep[276000/480370]
Loss: 1060.3329, lp_mean: 1060.3329, lp_var: 30738774016.0000, lp_std: 1753.2478
Epoch [6/1000], BatchStep[277000/480370]
Loss: 1256.1443, lp_mean: 1256.1443, lp_var: 43886829568.0000, lp_std: 2094.9182
Epoch [6/1000], BatchStep[278000/480370]
Loss: 2.3121, lp_mean: 2.3121, lp_var: 137931.6094, lp_std: 3.7139
Epoch [6/1000], BatchStep[279000/480370]
Loss: 567.4524, lp_mean: 567.4524, lp_var: 9470553088.0000, lp_std: 973.1676
Epoch [6/1000], BatchStep[280000/480370]
Loss: 0.5764, lp_mean: 0.5764, lp_var: 9830.6758, lp_std: 0.9915
Epoch [6/1000], BatchStep[281000/480370]
Loss: 6561.2144, lp_mean: 6561.2144, lp_var: 1233019338752.0000, lp_std: 11104.1406
Epoch [6/1000], BatchStep[282000/480370]
Loss: 165.9905, lp_mean: 165.9905, lp_var: 1026265664.0000, lp_std: 320.3538
Epoch [6/1000], BatchStep[283000/480370]
Loss: 503.1401, lp_mean: 503.1401, lp_var: 7380826624.0000, lp_std: 859.1174
Epoch [6/1000], BatchStep[284000/480370]
Loss: 663.6439, lp_mea

Epoch [6/1000], BatchStep[345000/480370]
Loss: 0.0000, lp_mean: 0.0000, lp_var: 0.0000, lp_std: 0.0000
Epoch [6/1000], BatchStep[346000/480370]
Loss: 2551.3284, lp_mean: 2551.3284, lp_var: 179866714112.0000, lp_std: 4241.0698
Epoch [6/1000], BatchStep[347000/480370]
Loss: 0.0000, lp_mean: 0.0000, lp_var: 0.0000, lp_std: 0.0000
Epoch [6/1000], BatchStep[348000/480370]
Loss: 1418.8635, lp_mean: 1418.8635, lp_var: 58952142848.0000, lp_std: 2428.0061
Epoch [6/1000], BatchStep[349000/480370]
Loss: 12340.6211, lp_mean: 12340.6211, lp_var: 4214523953152.0000, lp_std: 20529.3066
Epoch [6/1000], BatchStep[350000/480370]
Loss: 18813.5781, lp_mean: 18813.5781, lp_var: 9865144565760.0000, lp_std: 31408.8301
Epoch [6/1000], BatchStep[351000/480370]
Loss: 62028.4023, lp_mean: 62028.4023, lp_var: 106641328439296.0000, lp_std: 103267.2812
Epoch [6/1000], BatchStep[352000/480370]
Loss: 293.1350, lp_mean: 293.1350, lp_var: 2484990720.0000, lp_std: 498.4968
Epoch [6/1000], BatchStep[353000/480370]
Loss: 

Epoch [6/1000], BatchStep[414000/480370]
Loss: 44571.0312, lp_mean: 44571.0312, lp_var: 56834681470976.0000, lp_std: 75388.7812
Epoch [6/1000], BatchStep[415000/480370]
Loss: 53.6575, lp_mean: 53.6575, lp_var: 76244216.0000, lp_std: 87.3179
Epoch [6/1000], BatchStep[416000/480370]
Loss: 1653.3486, lp_mean: 1653.3486, lp_var: 44433625088.0000, lp_std: 2107.9285
Epoch [6/1000], BatchStep[417000/480370]
Loss: 1546.7876, lp_mean: 1546.7876, lp_var: 69083086848.0000, lp_std: 2628.3662
Epoch [6/1000], BatchStep[418000/480370]
Loss: 260.6662, lp_mean: 260.6662, lp_var: 1937515264.0000, lp_std: 440.1721
Epoch [6/1000], BatchStep[419000/480370]
Loss: 76.6644, lp_mean: 76.6644, lp_var: 152979312.0000, lp_std: 123.6848
Epoch [6/1000], BatchStep[420000/480370]
Loss: 241220.6719, lp_mean: 241220.6719, lp_var: 1670870992945152.0000, lp_std: 408762.9062
Epoch [6/1000], BatchStep[421000/480370]
Loss: 12.2609, lp_mean: 12.2609, lp_var: 3729221.7500, lp_std: 19.3112
Epoch [6/1000], BatchStep[422000/4803

Epoch [7/1000], BatchStep[3000/480370]
Loss: 0.0388, lp_mean: 0.0388, lp_var: 82.2268, lp_std: 0.0907
Epoch [7/1000], BatchStep[4000/480370]
Loss: 30.2753, lp_mean: 30.2753, lp_var: 26144754.0000, lp_std: 51.1319
Epoch [7/1000], BatchStep[5000/480370]
Loss: 4629.7622, lp_mean: 4629.7622, lp_var: 582515490816.0000, lp_std: 7632.2700
Epoch [7/1000], BatchStep[6000/480370]
Loss: 109.2323, lp_mean: 109.2323, lp_var: 308387104.0000, lp_std: 175.6095
Epoch [7/1000], BatchStep[7000/480370]
Loss: 59457.4062, lp_mean: 59457.4062, lp_var: 97470021369856.0000, lp_std: 98726.9141
Epoch [7/1000], BatchStep[8000/480370]
Loss: 1.4870, lp_mean: 1.4870, lp_var: 53629.3516, lp_std: 2.3158
Epoch [7/1000], BatchStep[9000/480370]
Loss: 4117.7163, lp_mean: 4117.7163, lp_var: 476918415360.0000, lp_std: 6905.9282
Epoch [7/1000], BatchStep[10000/480370]
Loss: 24.8521, lp_mean: 24.8521, lp_var: 19450542.0000, lp_std: 44.1028
Epoch [7/1000], BatchStep[11000/480370]
Loss: 1041.5684, lp_mean: 1041.5684, lp_var: 30

Epoch [7/1000], BatchStep[75000/480370]
Loss: 24817.5996, lp_mean: 24817.5996, lp_var: 16426479910912.0000, lp_std: 40529.5938
Epoch [7/1000], BatchStep[76000/480370]
Loss: 1609.7914, lp_mean: 1609.7914, lp_var: 71963172864.0000, lp_std: 2682.5955
Epoch [7/1000], BatchStep[77000/480370]
Loss: 61768.5859, lp_mean: 61768.5859, lp_var: 106592154419200.0000, lp_std: 103243.4766
Epoch [7/1000], BatchStep[78000/480370]
Loss: 0.0000, lp_mean: 0.0000, lp_var: 0.0000, lp_std: 0.0000
Epoch [7/1000], BatchStep[79000/480370]
Loss: 0.0000, lp_mean: 0.0000, lp_var: 0.0000, lp_std: 0.0000
Epoch [7/1000], BatchStep[80000/480370]
Loss: 30928.4434, lp_mean: 30928.4434, lp_var: 26045763813376.0000, lp_std: 51035.0508
Epoch [7/1000], BatchStep[81000/480370]
Loss: 107.3973, lp_mean: 107.3973, lp_var: 321031904.0000, lp_std: 179.1736
Epoch [7/1000], BatchStep[82000/480370]
Loss: 27707.0742, lp_mean: 27707.0742, lp_var: 21411718496256.0000, lp_std: 46272.7969
Epoch [7/1000], BatchStep[83000/480370]
Loss: 237

In [ ]:
# Exercise

In [ ]:
# w_real_deep
Epoch [1/1000] Test Loss: 144666329088.0000
Epoch [2/1000] Test Loss: 145073176576.0000
Epoch [3/1000] Test Loss: 146446860288.0000
Epoch [4/1000] Test Loss: 144554934272.0000
Epoch [5/1000] Test Loss: 145652776960.0000
Epoch [6/1000] Test Loss: 143921414144.0000

In [ ]:
# w_dropout_wo_loss_scaling
epoch [1/1000] Test Loss: 236042076160.0000
epoch [2/1000] Test Loss: 238885634048.0000 
Epoch [3/1000] Test Loss: 235835916288.0000
Epoch [4/1000] Test Loss: 237683245056.0000
Epoch [5/1000] Test Loss: 240484368384.0000
Epoch [6/1000] Test Loss: 235287920640.0000
Epoch [7/1000] Test Loss: 232444411904.0000
Epoch [8/1000] Test Loss: 233658007552.0000
Epoch [9/1000] Test Loss: 235095965696.0000
Epoch [10/1000] Test Loss: 232347713536.0000
Epoch [11/1000] Test Loss: 239848275968.0000
Epoch [12/1000] Test Loss: 242579668992.0000
Epoch [13/1000] Test Loss: 240169140224.0000
Epoch [14/1000] Test Loss: 239437922304.0000
